# About

this code is to condense the `clean_data\county_mrsa.csv` into a year-county format. right now, as each entry is a specific hospital-county-ear, there are duplicates when trying to join with the SDOH county-year dataset.

This will be done by taking the `Infections_Reported`, `Infections_Predicted `, `Patient_Days` columns, and pivoting them based off their `Hospital_Type` Value.

Note: hospital types are nativly listed as: ['Community (125-250 Beds)',
 'Community (<125 Beds)',
 'Major Teaching',
 'Rehabilitation Unit',
 'Critical Access',
 'Community (>250 Beds)',
 'LTAC',
 'Pediatric',
 'Free-Standing Rehabilitation Hospital',
 'Community, <125 Beds',
 'Community, >250 Beds',
 'Community, 125-250 Beds',
 'Long-Term Acute Care',
 'Free-Standing Rehabilitation'], so will need to combine columns to accoutn for column title discrepancies

Community hospitals are also renamed: Small, medium, large


Also combined 2020 H! and H2 into just "2020"

Data will be saved as `src/cse_8803_epi_project/data/mrsa_data/clean_data/county_year_mrsa.csv`

In [ ]:
import pandas as pd
mrsa_df = pd.read_csv('../../data/mrsa_data/clean_data/county_mrsa_df.csv')

In [3]:
# fix duplicates before aggregating
norm = {
    "Community, <125 Beds": "Community (<125 Beds)",
    "Community, 125-250 Beds": "Community (125-250 Beds)",
    "Community, >250 Beds": "Community (>250 Beds)",
    "LTAC": "Long-Term Acute Care",
    "Free-Standing Rehabilitation": "Free-Standing Rehabilitation Hospital",
}
mrsa_df['Hospital_Type'] = mrsa_df['Hospital_Type'].replace(norm)

norm = {
    "Community, <125 Beds": "cs",
    "Community, 125-250 Beds": "cm",
    "Community, >250 Beds": "cl",
    "Community (<125 Beds)": "cs",
    "Community (125-250 Beds)": "cm",
    "Community (>250 Beds)": "cl",
    "LTAC": "ltac",
    "Free-Standing Rehabilitation": "fsr",
}
mrsa_df['Hospital_Type'] = mrsa_df['Hospital_Type'].replace(norm)

# make sure the three fields are numeric
for col in ['Infections_Reported', 'Infections_Predicted', 'Patient_Days']:
    mrsa_df[col] = pd.to_numeric(mrsa_df[col], errors='coerce').fillna(0)

# pivot to get sums by hospital type within each county-year
county_years = mrsa_df.pivot_table(
    index=['Year', 'State', 'County'],
    columns='Hospital_Type',
    values=['Infections_Reported', 'Infections_Predicted', 'Patient_Days'],
    aggfunc='sum',
    fill_value=0
)

# flatten the MultiIndex columns: infections_reported__<Hospital_Type>, etc.
county_years.columns = [f"{metric.lower()}__{htype}" for metric, htype in county_years.columns]
county_years = county_years.reset_index()

county_years.head()

Year       State        County  infections_predicted__Critical Access  \
0  2017  California       Alameda                                   0.00   
1  2017  California        Amador                                   0.00   
2  2017  California         Butte                                   0.03   
3  2017  California     Calaveras                                   0.07   
4  2017  California  Contra Costa                                   0.00   

   infections_predicted__Free-Standing Rehabilitation Hospital  \
0                                               0.12             
1                                               0.00             
2                                               0.13             
3                                               0.00             
4                                               0.00             

   infections_predicted__Long-Term Acute Care  \
0                                        4.14   
1                                        0.00   
2                                        0.00   
3                                        0.00   
4                                        0.00   

   infections_predicted__Major Teaching  infections_predicted__Pediatric  \
0                                  7.22                             1.78   
1                                  0.00                             0.00   
2                                  0.00                             0.00   
3                                  0.00                             0.00   
4                                  9.74                             0.00   

   infections_predicted__Rehabilitation Unit  infections_predicted__cl  ...  \
0                                       0.34                      7.65  ...   
1                                       0.00                      0.00  ...   
2                                       0.00                      4.34  ...   
3                                       0.00                      0.00  ...   
4                                       0.14                      0.00  ...   

   infections_reported__cs  patient_days__Critical Access  \
0                      0.0                            0.0   
1                      0.0                            0.0   
2                      0.0                         1304.0   
3                      0.0                         3407.0   
4                      0.0                            0.0   

   patient_days__Free-Standing Rehabilitation Hospital  \
0                                             6549.0     
1                                                0.0     
2                                             7052.0     
3                                                0.0     
4                                                0.0     

   patient_days__Long-Term Acute Care  patient_days__Major Teaching  \
0                             23921.0                      139550.0   
1                                 0.0                           0.0   
2                                 0.0                           0.0   
3                                 0.0                           0.0   
4                                 0.0                      147700.0   

   patient_days__Pediatric  patient_days__Rehabilitation Unit  \
0                  46418.0                            17996.0   
1                      0.0                                0.0   
2                      0.0                                0.0   
3                      0.0                                0.0   
4                      0.0                             7251.0   

   patient_days__cl  patient_days__cm  patient_days__cs  
0          176986.0          148345.0           42859.0  
1               0.0               0.0            9275.0  
2           79120.0           37410.0           11772.0  
3               0.0               0.0               0.0  
4               0.0          159859.0           35029.0  

[5 rows x 30 columns]

In [ ]:
# group by year, state, and County to get overall county totals
county_totals = (
    mrsa_df.groupby(['Year', 'State', 'County'], as_index=False)
           [['Infections_Reported', 'Infections_Predicted', 'Patient_Days']]
           .sum()
           .rename(columns={
               'Infections_Reported': 'Infections_Reported_Total',
               'Infections_Predicted': 'Infections_Predicted_Total',
               'Patient_Days': 'Patient_Days_Total'
           })
)

# merge
county_years = county_years.merge(county_totals, on=['Year', 'State', 'County'], how='left')

# combinbe 2020H1 and 2020H2 into just 2020
county_years['Year'] = county_years['Year'].replace({'2020H1': '2020', '2020H2': '2020'})
numeric_cols = county_years.select_dtypes(include='number').columns
df_agg = county_years.groupby(['Year', 'State', 'County'], as_index=False)[numeric_cols].sum()
df_agg = df_agg.sort_values(['State', 'County', 'Year']).reset_index(drop=True)

df_agg['Year'].value_counts()



In [6]:
df_agg.to_csv('../../data/mrsa_data/clean_data/county_year_mrsa.csv')